# Install MMDetection & Byte Track

In [ ]:
!pip install torch==2.0.0+cu117 torchvision==0.15.1+cu117 --index-url https://download.pytorch.org/whl/cu117

!pip install -U openmim
!mim install mmengine
!mim install "mmcv-full"
!mim install "mmdet"
!git clone https://github.com/open-mmlab/mmdetection.git

In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack
!pip install -q loguru lap thop


In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

In [ ]:
!pip install numpy==1.23.1

# ByteTracker

INSTALLING **ByteTrack** via tutorial https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/how-to-track-and-count-vehicles-with-yolov8.ipynb?ref=blog.roboflow.com#scrollTo=8KdBkOflo2xY

In [ ]:
from IPython import display
display.clear_output()

import os
HOME = os.getcwd()
print(HOME)

import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

/content
yolox.__version__: 0.1.0


In [ ]:
# Initialize the ByteTrack tracker
from yolox.tracker.byte_tracker import BYTETracker
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
# Initialize the tracker
tracker = BYTETracker(BYTETrackerArgs)

# Google WebCam

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import base64


In [ ]:
from mmdet.apis import DetInferencer
import mmcv

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# RUNNING CAMERA TRACKER

In [ ]:
import cv2
from mmdet.apis import init_detector, inference_detector
import torch


In [ ]:
# Load the pre-trained YOLOX-S detector
config_file = '/content/mmdetection/configs/rtmdet/rtmdet_s_8xb32-300e_coco.py'
checkpoint_file = 'https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_s_8xb32-300e_coco/rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth'
device = 'cpu'  # Since you're using CPU
detector_model = init_detector(config_file, checkpoint_file, device=device)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_s_8xb32-300e_coco/rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth


Downloading: "https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_s_8xb32-300e_coco/rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth" to /root/.cache/torch/hub/checkpoints/rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth


The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



In [ ]:
def humanFinder(model_value):
    h_bounding_boxes = model_value.pred_instances.bboxes
    h_scores = model_value.pred_instances.scores
    h_labels = model_value.pred_instances.labels

    # Filter out detections where labels are not equal to 0
    bounding_boxes_and_scores = []
    for bbox, score, label in zip(h_bounding_boxes, h_scores, h_labels):
        if label == 0:
            result = torch.cat((bbox, score.unsqueeze(0)))
            bounding_boxes_and_scores.append(result)

    return bounding_boxes_and_scores

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# Start streaming video from webcam
video_stream()

# Label for video
label_html = 'Capturing...'

# Initialize bounding box to empty
bbox = ''
count = 0

while True:
    # Capture frame from the video stream
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    #detect objects in environment return all bounding boxes
    model_value = inference_detector(detector_model,img)

    #retrieve human bound box from frames
    bound_box = humanFinder(model_value)
    bound_box_1 = np.array([bound_box[0].tolist()])


    # Update tracker with the bounding box and scores
    online_targets = tracker.update(bound_box_1, img_info=img.shape, img_size=img.shape)

    # Get image dimensions
    height, width, _ = img.shape

    # Calculate center of the image
    center_x_img = width // 2
    center_y_img = height // 2

    print_T = ""

    # Create transparent overlay for bounding boxes
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    # Draw bounding boxes from the tracker
    for target in online_targets:
        bbox = target.tlwh
        cv2.rectangle(img,(int(bbox[0]),int (bbox[1])), (int(bbox[2]+int (bbox[1]) ), int(bbox[3])+int (bbox[1])), (0, 255, 0), 2)
        cv2.putText(img, f'Target ID: {target.track_id}', (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        bbox_array=cv2.rectangle(bbox_array,(int(bbox[0]),int (bbox[1])), (int(bbox[2]+int (bbox[1]) ), int(bbox[3])+int (bbox[1])), (0, 255, 0), 2)

        #Display Person ID
        bbox_array=cv2.putText(bbox_array, f'Target ID: {target.track_id}', (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        #Display camera movement instructions
        bbox_array=cv2.putText(bbox_array, f'Camera: {print_T}', (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Get image dimensions
        height, width, _ = img.shape

        # Calculate center of the image
        center_x_img = width // 2
        center_y_img = height // 2

        # Get center of the tracked person bounding box
        bbox_center_x = int(bbox[0] + bbox[2] / 2)
        bbox_center_y = int(bbox[1] + bbox[3] / 2)

        # Initialize variables to store camera movement directions and distances
        move_direction_x = ""
        move_direction_y = ""
        move_distance_x = 0
        move_distance_y = 0

        # Check if box center x coordinate is to the left or right of the center
        if bbox_center_x < center_x_img:
            move_direction_x = "left"
            move_distance_x = center_x_img - bbox_center_x
            print_x = f" move to the {move_direction_x} by {move_distance_x} pixels."

        elif bbox_center_x > center_x_img:
            move_direction_x = "right"
            move_distance_x = bbox_center_x - center_x_img
            print_x = f" move to the {move_direction_x} by {move_distance_x} pixels."



        # Check if box center y coordinate is above or below the center
        if bbox_center_y < center_y_img:
            move_direction_y = "up"
            move_distance_y = center_y_img - bbox_center_y
            print_y=f" and {move_direction_y} by {move_distance_y} pixels."

        elif bbox_center_y > center_y_img:
            move_direction_y = "down"
            move_distance_y = bbox_center_y - center_y_img
            print_y= f" and {move_direction_y} by {move_distance_y} pixels."

        bbox_array=cv2.putText(bbox_array, f'Camera:' + print_x + print_y, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)





    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes



<IPython.core.display.Javascript object>